# Báo Cáo Phân Tích Dự Án G1 Dual Arm Grasping RL (v22)

## 🎯 Mục Tiêu Báo Cáo

Báo cáo này phân tích chi tiết dự án **G1 Dual Arm Grasping RL (v22)**, một hệ thống huấn luyện robot hai tay thực hiện động tác nắm và nâng vật thể sử dụng Reinforcement Learning trong MuJoCo simulator.

**Mục tiêu phân tích:**
- ✅ Hiểu rõ cấu trúc dự án và các thành phần
- ✅ Phân tích chi tiết từng file code với code snippets đầy đủ
- ✅ Giải thích nguyên lý hoạt động và ý nghĩa của từng phần
- ✅ Nắm bắt luồng hoạt động tổng thể của hệ thống
- ✅ Hiểu cách tích hợp MuJoCo, Gymnasium, và Stable-Baselines3

---

## 📋 Mục Lục

1. [Giới Thiệu Dự Án](#1-giới-thiệu-dự-án)
2. [Cấu Trúc Thư Mục](#2-cấu-trúc-thư-mục)
3. [Phân Tích Chi Tiết Từng File](#3-phân-tích-chi-tiết-từng-file)
   - 3.1. Environment (g1_dual_arm_env_v22.py)
   - 3.2. Training Script (train_ppo_v22.py)
   - 3.3. Evaluation Script (eval_policy_v22.py)
   - 3.4. Demo Script (demo_pd_close_hand.py)
   - 3.5. Utilities (common.py)
4. [Nguyên Lý Hoạt Động Tổng Thể](#4-nguyên-lý-hoạt-động-tổng-thể)
5. [Kết Luận và Nhận Xét](#5-kết-luận-và-nhận-xét)


## 1. Giới Thiệu Dự Án

### 1.1. Tổng Quan

Dự án **G1 Dual Arm Grasping RL (v22)** là một hệ thống huấn luyện robot hai tay thực hiện động tác nắm và nâng vật thể (cube) sử dụng Reinforcement Learning (RL). Dự án sử dụng:

- **MuJoCo 3.x**: Physics simulator tiên tiến để mô phỏng vật lý robot
- **Gymnasium**: Framework chuẩn để tạo custom RL environments
- **Stable-Baselines3**: Thư viện RL hiện đại hỗ trợ PPO
- **PyTorch**: Deep learning framework cho neural networks

### 1.2. Tính Năng Chính

1. **Dual-arm robot**: Điều khiển 22 actuators (14 arm joints + 10 finger synergy actuators + 2 fist macros)
2. **Continuous grasping**: Synergy actuators điều khiển phối hợp 5 ngón tay mỗi tay để nắm vật thể
3. **Progress-based reward**: Reward dựa trên tiến trình (reach → contact → lift)
4. **Privileged observations**: Asymmetric observations cho critic (ground truth contact/poses)
5. **Domain randomization**: Randomize physics parameters (friction, size, solver) để tăng robustness
6. **Latency & sensor delay**: Mô phỏng độ trễ thực tế trong control và sensing
7. **Safety mechanisms**: Termination khi wrist velocity quá cao hoặc state invalid
8. **Curriculum learning**: Tự động điều chỉnh độ khó dựa trên success rate

### 1.3. Công Nghệ Sử Dụng

| Công Nghệ | Phiên Bản | Mục Đích |
|-----------|-----------|----------|
| **MuJoCo** | >=3.2.5 | Physics simulation engine |
| **Gymnasium** | >=0.29.1 | RL environment interface |
| **Stable-Baselines3** | >=2.3.2 | RL algorithms (PPO) |
| **PyTorch** | >=2.3 | Deep learning framework |
| **Python** | 3.10+ | Programming language |
| **NumPy** | >=1.26 | Numerical computations |

### 1.4. Mục Đích Ứng Dụng

Dự án này có thể được sử dụng để:
- Huấn luyện robot hai tay thực hiện manipulation tasks phức tạp
- Nghiên cứu về continuous control trong RL với privileged information
- Phát triển các ứng dụng robotic manipulation với contact-rich tasks
- So sánh hiệu quả giữa các reward shaping strategies


## 2. Cấu Trúc Thư Mục

### 2.1. Mô Tả Cấu Trúc Thư Mục

```
g1_dual_arm_grasping_rl/
│
├── envs/                          # Custom Gymnasium Environment
│   ├── __init__.py               # Package initialization
│   └── g1_dual_arm_env_v22.py    # ⭐ Main environment class (DualArmGraspEnvV22)
│
├── scripts/                       # Training và evaluation scripts
│   ├── train_ppo_v22.py          # ⭐ PPO training script với VecNormalize
│   ├── eval_policy_v22.py        # ⭐ Evaluation script cho trained models
│   └── demo_pd_close_hand.py     # Demo PD control (baseline)
│
├── utils/                         # Utility functions
│   └── common.py                 # Common argument parsers và helpers
│
├── models/                        # MuJoCo model files
│   ├── g1_dual_arm.xml           # ⭐ Main MuJoCo XML model
│   └── meshes/                   # STL mesh files cho robot parts
│
├── runs/                          # Training outputs (models, logs)
│   ├── ppo_dualarm_cuda_v22_prog_ft/  # PPO training outputs
│   │   ├── ppo_latest.zip        # Latest model checkpoint
│   │   ├── vecnorm_latest.pkl    # VecNormalize statistics
│   │   └── ppo_ckpt_*.zip        # Periodic checkpoints
│   └── ...
│
├── run_train.ps1                  # PowerShell script để khởi động training
├── run_eval.ps1                   # PowerShell script để đánh giá model
├── run_demo_pd.ps1                # PowerShell script để chạy demo
├── requirements.txt               # Python dependencies
└── README.md                      # Documentation
```

**Các file quan trọng nhất (được đánh dấu ⭐):**
1. **g1_dual_arm_env_v22.py**: Core environment class với progress-based reward
2. **train_ppo_v22.py**: Training script với VecNormalize và checkpointing
3. **eval_policy_v22.py**: Evaluation script với VecNormalize loading
4. **g1_dual_arm.xml**: MuJoCo physics model với dual-arm robot và hands


In [ ]:
import os
from pathlib import Path

# Lấy đường dẫn gốc của dự án
project_root = Path(".")
print("="*70)
print("  📁 CẤU TRÚC THƯ MỤC DỰ ÁN")
print("="*70)
print()

# Liệt kê cấu trúc thư mục
def print_tree(directory, prefix="", max_depth=3, current_depth=0):
    """In cấu trúc thư mục dạng tree"""
    if current_depth >= max_depth:
        return
    
    items = sorted(directory.iterdir())
    dirs = [item for item in items if item.is_dir() and not item.name.startswith('.')]
    files = [item for item in items if item.is_file()]
    
    # In thư mục trước
    for i, item in enumerate(dirs):
        is_last_dir = (i == len(dirs) - 1) and len(files) == 0
        print(f"{prefix}{'└──' if is_last_dir else '├──'} {item.name}/")
        new_prefix = prefix + ("    " if is_last_dir else "│   ")
        print_tree(item, new_prefix, max_depth, current_depth + 1)
    
    # In file sau
    for i, item in enumerate(files):
        is_last = i == len(files) - 1
        print(f"{prefix}{'└──' if is_last else '├──'} {item.name}")

print_tree(project_root)

# Thống kê số lượng file
print("\n" + "="*70)
print("  📊 THỐNG KÊ DỰ ÁN")
print("="*70)

def count_files(directory, extensions=None):
    """Đếm số file theo extension"""
    count = 0
    for root, dirs, files in os.walk(directory):
        # Bỏ qua các thư mục ẩn
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        for file in files:
            if extensions is None or any(file.endswith(ext) for ext in extensions):
                count += 1
    return count

py_files = count_files(project_root, ['.py'])
xml_files = count_files(project_root, ['.xml'])
stl_files = count_files(project_root, ['.stl'])

print(f"Python files (.py): {py_files}")
print(f"XML model files (.xml): {xml_files}")
print(f"STL mesh files (.stl): {stl_files}")
print(f"Total Python files: {py_files}")


## 3. Phân Tích Chi Tiết Từng File

### 3.1. File: `envs/g1_dual_arm_env_v22.py` - Core Environment

Đây là file **quan trọng nhất** của dự án, định nghĩa custom Gymnasium environment cho robot G1 Dual Arm với progress-based reward và privileged observations.


#### 3.1.1. Class DualArmGraspEnvV22 - Khởi Tạo

**Code quan trọng:**

```python
class DualArmGraspEnvV22(gym.Env):
    metadata = {"render_modes": ["human","none"], "render_fps": 60}

    def __init__(
        self,
        xml_path: str = DEFAULT_XML,
        render_mode: str | None = None,
        max_steps: int = 400,
        use_privileged: bool = True,
        obs_noise_std: float = 0.004,
        latency_steps: int = 2,
        sensor_delay_steps: int = 1,
        hold_steps: int = 120,
        slip_torque_lim: float = 45.0,
        max_wrist_angvel: float = 20.0,
        strict_check: bool = True,
        seed: int | None = None,
    ):
        super().__init__()
        
        # Load MuJoCo model
        self.model = mj.MjModel.from_xml_path(self.xml_path)
        self.data  = mj.MjData(self.model)
        
        # Map actuators (22 total: 14 arm + 10 finger + 2 fist)
        self.actuator_names = ARM_ACTS + FINGER_SYNS + FIST_MACROS
        self.aid = np.array([_safe_name2id(self.model, mj.mjtObj.mjOBJ_ACTUATOR, n)
                             for n in self.actuator_names], dtype=int)
        
        # Action space: 22D (normalized [-1, 1])
        n_act = len(self.actuator_names)
        self.action_space = spaces.Box(low=-np.ones(n_act, np.float32),
                                       high= np.ones(n_act, np.float32),
                                       dtype=np.float32)
        
        # Observation space: dynamic based on sensors + joints + privileged
        base_dim = 0
        for sid in self.sids:
            base_dim += int(self.model.sensor_dim[sid]) if sid >= 0 else 1
        base_dim += len(self.jids)*2  # joint pos + vel
        priv_dim = sum(dim for _, dim in self._priv_items) if self.use_privileged else 0
        self.obs_dim = base_dim + priv_dim
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(self.obs_dim,), dtype=np.float32)
```

**Giải thích chi tiết:**

1. **Kế thừa từ `gym.Env`**: 
   - Tuân thủ Gymnasium API chuẩn
   - Cho phép tích hợp với Stable-Baselines3

2. **MuJoCo Model Loading**:
   - `MjModel.from_xml_path()`: Load XML model file vào MuJoCo
   - `MjData()`: Tạo data structure chứa state (positions, velocities, controls)

3. **Actuators (22 total)**:
   - **ARM_ACTS (14)**: 7 joints mỗi tay (shoulder pitch/roll/yaw, elbow, wrist roll/pitch/yaw)
   - **FINGER_SYNS (10)**: 5 ngón mỗi tay (thumb, index, middle, ring, little)
   - **FIST_MACROS (2)**: left_fist, right_fist (macro actuators)

4. **Action Space (22D)**:
   - Range: [-1.0, 1.0] (normalized)
   - Mapped to actuator control ranges trong `_set_action()`

5. **Observation Space (Dynamic)**:
   - **Base sensors**: EE positions, quaternions, touch sensors, wrist velocities, palm forces/torques
   - **Joint states**: 14 arm joints × 2 (pos + vel) = 28D
   - **Privileged (optional)**: Cube qpos/qvel (13D), contact force sum (1D), plate normal z (1D)
   - **Total**: ~50-70D tùy sensors có trong XML

6. **Key Parameters**:
   - `use_privileged`: Nếu True, thêm ground truth info cho critic (asymmetric obs)
   - `obs_noise_std`: Gaussian noise cho observations (0.004)
   - `latency_steps`: Độ trễ control (2 steps)
   - `sensor_delay_steps`: Độ trễ sensor (1 step)
   - `hold_steps`: Số steps phải giữ thành công để hoàn thành task (120)
   - `slip_torque_lim`: Ngưỡng torque để phát hiện slip (45.0 N⋅m)
   - `max_wrist_angvel`: Ngưỡng angular velocity để safety termination (20.0 rad/s)


#### 3.1.2. Observation Function (_obs_now)

**Code:**

```python
def _obs_now(self):
    arr = []
    # Sensor data
    for sid in self.sids:
        v = self._sensor_vec(sid)
        if v.size == 0: arr.append(0.0)
        else:           arr.extend(v.tolist())
    # Joint states (pos + vel)
    q, v = self._arm_state()
    arr.extend(q.tolist()); arr.extend(v.tolist())
    # Privileged observations (if enabled)
    if self.use_privileged: arr.extend(self._priv_obs().tolist())
    return np.asarray(arr, dtype=np.float64)

def _priv_obs(self):
    out = []
    # Cube state (qpos 7D + qvel 6D = 13D)
    if self._jid_cube_free >= 0:
        qadr = int(self.model.jnt_qposadr[self._jid_cube_free])
        dadr = int(self.model.jnt_dofadr[self._jid_cube_free])
        out.extend(self.data.qpos[qadr:qadr+7].tolist())
        out.extend(self.data.qvel[dadr:dadr+6].tolist())
    else:
        out.extend([0.0]*13)
    # Contact force sum (1D)
    touch_sum = 0.0
    for sid in self._touch_sids:
        v = self._sensor_vec(sid); touch_sum += float(v[0]) if v.size else 0.0
    out.append(touch_sum)
    # Plate normal z (1D)
    pz = 0.0
    if self._sid_plate_top >= 0:
        p = self._sensor_vec(self._sid_plate_top)
        if p.size >= 3: pz = float(p[2])
    out.append(pz)
    return np.asarray(out, dtype=np.float64)
```

**Giải thích:**

1. **Sensor data**: Lấy từ MuJoCo sensors (EE positions, quaternions, touch, velocities, forces)
2. **Joint states**: 14 arm joints × 2 (position + velocity) = 28D
3. **Privileged observations** (chỉ cho critic, không cho actor):
   - **Cube state**: Ground truth position/orientation và velocity (13D)
   - **Contact force sum**: Tổng lực tiếp xúc từ 5 fingertips (1D)
   - **Plate normal z**: Z coordinate của plate top (1D)
4. **Noise augmentation**: Thêm Gaussian noise với std=0.004 vào observations
5. **Sensor delay**: Observations được delay bởi `sensor_delay_steps` (ring buffer)

**Ý nghĩa**: Observation này cung cấp đầy đủ thông tin về trạng thái robot và vật thể để agent có thể đưa ra quyết định điều khiển phù hợp. Privileged info giúp critic học tốt hơn trong khi actor chỉ dùng public sensors.


#### 3.1.3. Action Application Function (_set_action)

**Code:**

```python
def _set_action(self, action):
    a = np.clip(np.asarray(action, np.float64), -1.0, 1.0)
    u = self._smooth_and_limit(a)
    self._jerk = float(np.linalg.norm(u - self._u_prev))
    u_lat = self._apply_latency(u)
    self._u_prev = u.copy()
    for i, aid in enumerate(self.aid):
        if aid < 0: continue
        lo, hi = self.model.actuator_ctrlrange[aid]
        self.data.ctrl[aid] = lo + 0.5*(u_lat[i]+1.0)*(hi-lo)

def _smooth_and_limit(self, a):
    n_arm = len(ARM_ACTS)
    arm, fin = a[:n_arm].copy(), a[n_arm:].copy()
    # Exponential smoothing
    arm = self._alpha_arm*arm + (1-self._alpha_arm)*self._u_prev[:n_arm]
    fin = self._alpha_finger*fin + (1-self._alpha_finger)*self._u_prev[n_arm:]
    # Rate limiting
    def limit(x, xprev, du): return xprev + np.clip(x-xprev, -du, du)
    arm = limit(arm, self._u_prev[:n_arm], self._du_max_arm)
    fin = limit(fin, self._u_prev[n_arm:], self._du_max_finger)
    return np.concatenate([arm, fin])

def _apply_latency(self, u):
    self._ring_idx = (self._ring_idx + 1) % len(self._u_ring)
    self._u_ring[self._ring_idx] = u.copy()
    delayed_idx = (self._ring_idx - self.latency_steps) % len(self._u_ring)
    return self._u_ring[delayed_idx]
```

**Giải thích chi tiết:**

1. **Action preprocessing**:
   - Clip về range [-1.0, 1.0]
   - **Smoothing**: Exponential moving average (alpha_arm=0.4, alpha_finger=0.22)
   - **Rate limiting**: Giới hạn thay đổi mỗi step (du_max_arm=0.15, du_max_finger=0.08)
   - **Jerk calculation**: Tính độ "giật" của control signal

2. **Latency simulation**:
   - Ring buffer để delay action bởi `latency_steps` (mặc định 2 steps)
   - Mô phỏng độ trễ thực tế trong control loop

3. **Control mapping**:
   - Map từ normalized action [-1, 1] → actuator control range [lo, hi]
   - Formula: `ctrl = lo + 0.5*(action+1.0)*(hi-lo)`

4. **Grip reflex** (trong `step()`):
   - Tự động điều chỉnh `right_fist` dựa trên touch và torque
   - Nếu touch > 0.02 và torque < slip_limit → tăng grip
   - Nếu torque > slip_limit*2.2 → giảm grip (tránh slip)

**Ý nghĩa**: Hàm này chuyển đổi normalized actions từ agent thành các lệnh điều khiển thực tế cho robot với smoothing, rate limiting, và latency để mô phỏng điều kiện thực tế.


#### 3.1.4. Reward Function (_compute_reward)

**Code:**

```python
def _compute_reward(self, M):
    # Progress-based rewards
    r_reach_prog = 2.0 * (self._prev_dist - M["dist"])
    gate_contact = 1.0 if M["touch_sum"] > 0.03 else 0.0
    r_lift_prog = 6.0 * (M["dz_rel"] - self._prev_dz_rel) * gate_contact
    r_lift_prog = float(np.clip(r_lift_prog, -0.5, 1.0))
    
    # Contact reward
    r_touch = 0.4*np.tanh(0.6*M["touch_sum"]) + 0.4*(M["nf"]>=2)*gate_contact
    
    # Penalties
    a_ctrl = self.data.ctrl[self.aid[self.aid >= 0]]
    r_pen = -0.0015*M["wrist_rate"] \
            -0.0030*self._jerk \
            -0.0020*(float(np.linalg.norm(a_ctrl)) if a_ctrl.size>0 else 0.0) \
            -0.0025*max(0.0, M["tau_mag"] - self.slip_torque_lim)
    r_step = -0.002
    
    reward = r_reach_prog + r_lift_prog + r_touch + r_pen + r_step
    
    # Success condition
    success = 1.0 if (M["dz_rel"] > 0.04 and M["touch_sum"] > 0.08) else 0.0
    if success > 0.5: self._hold_counter += 1
    else:             self._hold_counter = 0
    if self._hold_counter >= self.hold_steps_required:
        reward += 10.0  # Bonus for stable hold
    
    info = {"success":success, "hold": float(self._hold_counter >= self.hold_steps_required)}
    info.update(M)
    self._prev_dist = M["dist"]; self._prev_dz_rel = M["dz_rel"]
    return reward, info
```

**Giải thích chi tiết:**

1. **Progress-based rewards**:
   - **r_reach_prog**: Reward khi giảm khoảng cách đến cube (2.0 × distance reduction)
   - **r_lift_prog**: Reward khi nâng cube lên (6.0 × height increase), chỉ khi có contact (gated)
   - **Gate contact**: Chỉ cho reward lift khi `touch_sum > 0.03` (đã tiếp xúc)

2. **Contact reward**:
   - **r_touch**: Reward dựa trên tổng lực tiếp xúc (tanh để saturate) + bonus nếu ≥2 ngón chạm

3. **Penalties**:
   - **Wrist rate**: Penalty cho angular velocity cao (tránh quay nhanh)
   - **Jerk**: Penalty cho control signal giật (smoothness)
   - **Control magnitude**: Penalty cho effort (energy efficiency)
   - **Slip torque**: Penalty khi torque vượt ngưỡng slip (tránh làm rơi)

4. **Step penalty**: -0.002 mỗi step (khuyến khích hoàn thành nhanh)

5. **Success condition**:
   - `dz_rel > 0.04`: Cube được nâng lên ít nhất 4cm
   - `touch_sum > 0.08`: Có đủ lực tiếp xúc
   - **Hold bonus**: +10.0 reward nếu giữ thành công trong `hold_steps` (120 steps)

**Ý nghĩa**: Reward function này khuyến khích robot học cách tiếp cận, nắm, và nâng vật thể một cách ổn định, với penalties để đảm bảo smooth và safe motions.


#### 3.1.5. Domain Randomization (_randomize_physics)

**Code:**

```python
def _randomize_physics(self):
    L = float(np.clip(self.curr_level, 0.0, 1.0))  # Curriculum level
    gid = _safe_name2id(self.model, mj.mjtObj.mjOBJ_GEOM, "grasp_cube_geom")
    if gid >= 0:
        # Randomize cube size
        sz = self.model.geom_size[gid].copy()
        noise = self._rng.uniform(-0.003*L, 0.003*L, size=3)
        self.model.geom_size[gid] = np.clip(sz + noise, 0.012, 0.030)
        # Randomize cube friction
        fr = self.model.geom_friction[gid].copy()
        fr *= (1.0 + self._rng.uniform(-0.3*L, 0.3*L, size=3))
        fr[0] = np.clip(fr[0], 0.5, 5.0); fr[1] = np.clip(fr[1], 0.005, 0.10); fr[2] = np.clip(fr[2], 0.002, 0.05)
        self.model.geom_friction[gid] = fr
    
    # Randomize fingertip pad friction
    names = [mj.mj_id2name(self.model, mj.mjtObj.mjOBJ_GEOM, i) for i in range(self.model.ngeom)]
    for i, n in enumerate(names):
        if n and n.endswith("_tip_pad"):
            fr = self.model.geom_friction[i].copy()
            fr *= (1.0 + self._rng.uniform(-0.25*L, 0.25*L, size=3))
            fr[0] = np.clip(fr[0], 3.0, 12.0); fr[1] = np.clip(fr[1], 0.02, 0.20); fr[2] = np.clip(fr[2], 0.002, 0.05)
            self.model.geom_friction[i] = fr
    
    # Randomize solver parameters
    def jitter_sol(i, sref_scale=0.15*L, simp_scale=0.005*L):
        if hasattr(self.model, "geom_solref"):
            sr = self.model.geom_solref[i].copy()
            sr[0] *= 1.0 + self._rng.uniform(-sref_scale, sref_scale)
            sr[1] *= 1.0 + self._rng.uniform(-sref_scale, sref_scale)
            self.model.geom_solref[i] = sr
        if hasattr(self.model, "geom_solimp"):
            si = self.model.geom_solimp[i].copy()
            si[:2] = np.clip(si[:2] + self._rng.uniform(-simp_scale, simp_scale, size=2), 0.90, 0.999)
            self.model.geom_solimp[i] = si
    
    # Randomize actuator ranges
    for aid in self.aid:
        if aid < 0: continue
        lo, hi = self.model.actuator_ctrlrange[aid]
        span = (hi - lo) * (1.0 + self._rng.uniform(-0.10*L, 0.10*L))
        self.model.actuator_ctrlrange[aid] = np.array([lo, lo+span], dtype=np.float64)
```

**Giải thích:**

1. **Curriculum level (L)**: Tăng dần từ 0.0 → 1.0 dựa trên success rate
   - Nếu `succ_ema > 0.35`: Tăng level (+0.002)
   - Nếu `succ_ema <= 0.35`: Giảm level (-0.001)

2. **Cube randomization**:
   - **Size**: ±0.003*L (tối đa ±3mm khi L=1.0)
   - **Friction**: ±30%*L (tối đa ±30% khi L=1.0)

3. **Fingertip pad friction**: ±25%*L để mô phỏng wear/conditions khác nhau

4. **Solver parameters**: Randomize `solref` và `solimp` để mô phỏng contact dynamics khác nhau

5. **Actuator ranges**: ±10%*L để mô phỏng actuator variations

**Ý nghĩa**: Domain randomization giúp policy robust hơn với variations trong physics parameters, tăng khả năng transfer sang real robot.


#### 3.1.6. Gymnasium API Methods (reset, step)

**Code:**

```python
def reset(self, *, seed=None, options=None):
    if seed is not None: self._rng = np.random.default_rng(seed)
    if options and options.get("xml_path") and options["xml_path"] != self.xml_path:
        self.xml_path = options["xml_path"]
        self.model = mj.MjModel.from_xml_path(self.xml_path); self.data = mj.MjData(self.model)
        if self._viewer is not None:
            try: self._viewer.close()
            except: pass
            self._viewer = None
    self._reset_internal(randomize=True)
    obs = self._apply_obs_noise(self._obs_now())
    self._ring_idx = (self._ring_idx + 1) % len(self._obs_ring)
    self._obs_ring[self._ring_idx] = obs
    delayed_idx = (self._ring_idx - self.sensor_delay_steps) % len(self._obs_ring)
    return self._obs_ring[delayed_idx].astype(np.float32), {}

def step(self, action):
    # Rate limit arm joint velocities
    for j in self.jids:
        if j >= 0:
            dadr = int(self.model.jnt_dofadr[j])
            self.data.qvel[dadr] = float(np.clip(self.data.qvel[dadr], -self._max_arm_qvel, self._max_arm_qvel))
    
    self._set_action(action)
    
    # Grip reflex (automatic fist adjustment)
    # ... (code đã trình bày ở trên)
    
    mj.mj_step(self.model, self.data); self.step_count += 1
    
    # Get observation with delay
    obs_now = self._apply_obs_noise(self._obs_now())
    self._ring_idx = (self._ring_idx + 1) % len(self._obs_ring)
    self._obs_ring[self._ring_idx] = obs_now
    delayed_idx = (self._ring_idx - self.sensor_delay_steps) % len(self._obs_ring)
    obs = self._obs_ring[delayed_idx].astype(np.float32)
    
    # Compute reward
    M = self._compute_metrics()
    reward, info = self._compute_reward(M)
    
    # Update curriculum
    self._succ_ema = self._ema_beta*self._succ_ema + (1-self._ema_beta)*info["success"]
    if self._succ_ema > 0.35: self.curr_level = float(np.clip(self.curr_level + 0.002, 0.0, 1.0))
    else:                      self.curr_level = float(np.clip(self.curr_level - 0.001, 0.0, 1.0))
    if self.curr_level > 0.15: self.curr_phase = 1
    if self.curr_level > 0.45: self.curr_phase = 2
    
    # Check termination
    terminated = False; truncated = False
    unsafe, reason = self._safety_termination(M)
    if unsafe:
        terminated = True; info["termination_reason"] = reason
    if info["hold"] > 0.5:
        terminated = True; info["termination_reason"] = "success_hold"
    if self.step_count >= self.max_steps: truncated = True
    
    if self.render_mode == "human" and self._viewer is not None:
        self._viewer.sync()
    
    info["curr_level"] = self.curr_level; info["succ_ema"] = self._succ_ema
    info["phase"] = self.curr_phase; info["jerk"] = self._jerk
    return obs, reward, terminated, truncated, info
```

**Giải thích:**

1. **reset()**:
   - Reset environment về trạng thái ban đầu
   - Randomize physics parameters
   - Place cube trên plate với random offset
   - Return delayed observation (sensor delay)

2. **step()**:
   - **Rate limit**: Clip arm joint velocities để tránh quá nhanh
   - **Apply action**: Gọi `_set_action()` với smoothing, rate limiting, latency
   - **Grip reflex**: Tự động điều chỉnh fist dựa trên touch/torque
   - **Physics step**: `mj.mj_step()` - tiến simulation 1 bước
   - **Get observation**: Với sensor delay (ring buffer)
   - **Compute reward**: Progress-based với gating
   - **Update curriculum**: Tự động điều chỉnh difficulty level
   - **Check termination**:
     - `terminated`: Safety violation hoặc success hold
     - `truncated`: Max steps reached
   - **Return**: (obs, rew, terminated, truncated, info)

**Ý nghĩa**: Tuân thủ Gymnasium API chuẩn, với các tính năng nâng cao như curriculum learning, safety checks, và realistic delays.


### 3.2. File: `scripts/train_ppo_v22.py` - PPO Training Script

Phân tích script huấn luyện với thuật toán PPO (Proximal Policy Optimization) và VecNormalize.


#### 3.2.1. VecNormalize Setup

**Code:**

```python
def make_vec(n_env, xml, use_priv, max_steps):
    def _thunk(seed):
        return lambda: make_env(xml_path=xml, render=False, use_privileged=use_priv, max_steps=max_steps, seed=seed)
    env_fns = [_thunk(1234+i) for i in range(max(1, n_env))]
    venv_raw = DummyVecEnv(env_fns) if n_env==1 else SubprocVecEnv(env_fns)
    venv = VecNormalize(venv_raw, norm_obs=True, norm_reward=True, clip_obs=10.0, gamma=0.995)
    return venv
```

**Giải thích:**

1. **Vectorized environments**:
   - `DummyVecEnv`: Nếu chỉ 1 env (sequential)
   - `SubprocVecEnv`: Nếu nhiều env (parallel, mỗi env trong process riêng)

2. **VecNormalize**:
   - **norm_obs=True**: Normalize observations (running mean/std)
   - **norm_reward=True**: Normalize rewards (running mean/std với gamma=0.995)
   - **clip_obs=10.0**: Clip observations về [-10, 10] để tránh outliers
   - **gamma=0.995**: Discount factor cho reward normalization

3. **Lưu VecNormalize stats**:
   - File `vecnorm_latest.pkl` chứa running statistics
   - **Quan trọng**: Phải load cùng stats khi evaluation!

**Ý nghĩa**: VecNormalize giúp stabilize training bằng cách normalize observations và rewards, giảm variance và tăng convergence speed.


#### 3.2.2. PPO Model Configuration

**Code:**

```python
policy_kwargs = dict(activation_fn=torch.nn.Tanh, net_arch=[dict(pi=[256,256], vf=[256,256])])
model = PPO("MlpPolicy", venv, n_steps=args.rollout, batch_size= args.rollout*max(1,args.n_env)//4,
            learning_rate=2.5e-4, gamma=0.995, gae_lambda=0.95, clip_range=0.2, ent_coef=0.0,
            verbose=1, device=args.device, policy_kwargs=policy_kwargs)
```

**Giải thích chi tiết các hyperparameters:**

1. **Policy**: `"MlpPolicy"` - Multi-layer perceptron (fully connected neural network)

2. **Network architecture**:
   - **Policy network (pi)**: [256, 256] hidden layers
   - **Value network (vf)**: [256, 256] hidden layers
   - **Activation**: Tanh (smooth, bounded)

3. **Learning rate**: `2.5e-4` (0.00025)
   - Standard cho PPO
   - Có thể dùng learning rate schedule nếu cần

4. **n_steps**: `args.rollout` (mặc định 1024)
   - Số steps mỗi rollout trước khi update policy
   - Lớn hơn → more stable nhưng slower

5. **batch_size**: `rollout * n_env // 4`
   - Ví dụ: 1024 * 4 // 4 = 1024
   - Batch size cho training

6. **gamma**: `0.995`
   - Discount factor cho future rewards
   - Gần 1.0 → focus vào long-term rewards

7. **gae_lambda**: `0.95`
   - GAE (Generalized Advantage Estimation) lambda
   - Bias-variance tradeoff cho advantage estimation

8. **clip_range**: `0.2`
   - PPO clipping range
   - Giới hạn policy update để tránh quá lớn

9. **ent_coef**: `0.0`
   - Entropy coefficient (exploration bonus)
   - 0.0 = không có entropy bonus (có thể dùng curriculum thay thế)

10. **Resume training**:
    ```python
    if args.resume:
        model = PPO.load(args.resume, env=venv, device=args.device)
        if args.reset_vf:
            reinit_value_net(model)  # Reset value network
    ```

11. **Checkpoint callback**:
    ```python
    ckpt_cb = CheckpointCallback(
        save_freq=max(1, args.save_every)//max(1,args.n_env),
        save_path=logdir, name_prefix="ppo_ckpt",
        save_replay_buffer=False, save_vecnormalize=False
    )
    ```

**Ý nghĩa**: Configuration này được tối ưu cho on-policy learning với PPO, sử dụng VecNormalize để stabilize training và checkpointing để resume training.


### 3.3. File: `scripts/eval_policy_v22.py` - Evaluation Script

Phân tích script đánh giá trained models.


#### 3.3.1. Evaluation Logic

**Code:**

```python
def main():
    args = parse()
    
    def _mk():
        return make_env(xml_path=args.xml,
                        render=args.render,
                        use_privileged=args.use_priv,
                        max_steps=args.max_steps)
    
    # Load VecNormalize stats
    venv_raw = DummyVecEnv([_mk])
    venv = VecNormalize.load(args.venv_stats, venv_raw)
    venv.training = False  # Không update stats khi eval
    venv.norm_reward = False  # Không normalize reward khi eval
    
    model = PPO.load(args.ckpt)
    
    rets, succs, holds = [], [], []
    for ep in range(args.episodes):
        obs = venv.reset()
        ep_ret, steps = 0.0, 0
        
        while True:
            action, _ = model.predict(obs, deterministic=args.deterministic)
            obs, rew, done, infos = venv.step(action)
            ep_ret += float(rew[0])
            steps  += 1
            
            info = infos[0] if isinstance(infos, (list, tuple)) else infos
            if bool(done[0]):
                s = int(info.get("success", 0))
                h = int(info.get("hold", 0))
                print(f"[EP {ep:02d}] ret={ep_ret:.2f} | len={steps} | success={s} | hold={h} | curr_level={info.get('curr_level',0):.2f}")
                rets.append(ep_ret); succs.append(s); holds.append(h)
                break
    
    print("\n========== EVAL SUMMARY ==========")
    print(f"Episodes           : {len(rets)}")
    print(f"Mean reward        : {np.mean(rets):.2f} ± {np.std(rets):.2f}")
    print(f"Success rate       : {100.0*sum(succs)/len(succs):.1f}%")
    print(f"Hold (stable) rate : {100.0*sum(holds)/len(holds):.1f}%")
```

**Giải thích:**

1. **Load VecNormalize stats**:
   - **Quan trọng**: Phải load cùng `vecnorm_latest.pkl` từ training run
   - `venv.training = False`: Không update running stats khi eval
   - `venv.norm_reward = False`: Không normalize reward (để xem raw reward)

2. **Load model**: `PPO.load(args.ckpt)` - Load checkpoint

3. **Evaluation loop**:
   - Reset environment mỗi episode
   - Predict action với `deterministic=True/False`
   - Accumulate reward và track metrics
   - Collect: total reward, success rate, hold rate

4. **Metrics**:
   - **Mean reward**: Trung bình reward qua episodes
   - **Success rate**: % episodes đạt success condition (dz_rel > 0.04, touch > 0.08)
   - **Hold rate**: % episodes giữ thành công trong `hold_steps` (120 steps)

**Ý nghĩa**: Script này cung cấp tool để evaluate và so sánh trained models một cách chuẩn hóa, với metrics rõ ràng về performance.


### 3.4. File: `scripts/demo_pd_close_hand.py` - PD Control Demo

File này demo PD control để đóng nắm tay (baseline, không dùng RL).

**Lưu ý**: File này có import sai (cần sửa `from envs.g1_dual_arm_env import DualArmGraspEnv` → `from envs.g1_dual_arm_env_v22 import DualArmGraspEnvV22`).

**Code:**

```python
import argparse, time, numpy as np
from envs.g1_dual_arm_env_v22 import DualArmGraspEnvV22

if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("--xml", type=str, default="models/g1_dual_arm.xml")
    ap.add_argument("--secs", type=float, default=6.0)
    ap.add_argument("--fist", action="store_true")
    args = ap.parse_args()
    
    env = DualArmGraspEnvV22(xml_path=args.xml, render_mode="human")
    obs, _ = env.reset()
    
    a = np.zeros(env.action_space.shape[0], dtype=np.float32)
    if args.fist:
        a[-2:] = 1.0  # left_fist, right_fist
    
    t_end = time.time() + float(args.secs)
    while time.time() < t_end:
        obs, rew, term, trunc, info = env.step(a)
        if term or trunc:
            env.reset()
    env.close()
```

**Giải thích:**

1. **Mục đích**: Demo đơn giản để test MuJoCo viewer và XML model
2. **Action**: Zero action (robot ở rest pose), hoặc activate fist nếu `--fist`
3. **Loop**: Chạy trong `--secs` giây, reset nếu episode kết thúc

**Ý nghĩa**: Baseline để so sánh với RL policy, và để test viewer/model trước khi training.


### 3.5. File: `utils/common.py` - Common Utilities

File này chứa các helper functions cho argument parsing.

**Code:**

```python
import os, argparse

def add_xml_arg(p):
    p.add_argument("--xml", type=str, default="models/g1_dual_arm.xml", help="Path to MuJoCo XML")
    return p

def add_train_args(p):
    p.add_argument("--total-steps", type=int, default=400_000)
    p.add_argument("--lr", type=float, default=3e-4)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--save-every", type=int, default=100_000)
    p.add_argument("--run-name", type=str, default=None)
    p.add_argument("--render", action="store_true")
    return p

def make_run_dir(prefix, run_name=None):
    import time, pathlib
    if run_name is None:
        run_name = time.strftime("%Y%m%d-%H%M%S")
    path = pathlib.Path("runs")/f"{prefix}_{run_name}"
    path.mkdir(parents=True, exist_ok=True)
    return str(path)
```

**Giải thích:**

1. **add_xml_arg()**: Helper để thêm XML path argument
2. **add_train_args()**: Helper để thêm training arguments
3. **make_run_dir()**: Tạo thư mục run với timestamp nếu không có tên

**Ý nghĩa**: Utilities để tái sử dụng code và giữ consistency giữa các scripts.


## 4. Nguyên Lý Hoạt Động Tổng Thể

### 4.1. Luồng Hoạt Động Tổng Quan

```
Start Training
    ↓
Load XML Model (g1_dual_arm.xml)
    ↓
Create VecEnv với VecNormalize
    ↓
Initialize PPO Model
    ↓
For each training iteration:
    ├─ Collect Rollouts (n_steps × n_env)
    ├─ Compute Advantages (GAE)
    ├─ Update Policy (multiple epochs)
    ├─ Update Value Function
    ├─ Save Checkpoint (periodic)
    └─ Update VecNormalize Stats
    ↓
Save Final Model + VecNormalize Stats
    ↓
Evaluation với eval_policy_v22.py
```

### 4.2. Chi Tiết Luồng Training

#### 4.2.1. PPO Training Flow

```
1. Initialize PPO model với MlpPolicy
   - Policy network: [256, 256] hidden layers
   - Value network: [256, 256] hidden layers
   - Activation: Tanh

2. Create VecEnv với VecNormalize
   - Parallel environments (SubprocVecEnv nếu n_env > 1)
   - Normalize observations và rewards
   - Running statistics được update mỗi step

3. For each iteration:
   a. Collect 1024 steps × n_env (rollout)
   b. Compute advantages với GAE (lambda=0.95)
   c. Update policy và value function:
      - Batch size: rollout * n_env // 4
      - Clip range: 0.2
      - Learning rate: 2.5e-4
   d. Update VecNormalize statistics
   e. Save checkpoint mỗi save_every steps

4. Save final model và VecNormalize stats
   - ppo_latest.zip
   - vecnorm_latest.pkl
```

### 4.3. Environment Interaction Flow

```
Agent → Action (22D normalized [-1, 1])
    ↓
_set_action():
  - Clip action về [-1, 1]
  - Smoothing (exponential moving average)
  - Rate limiting (du_max_arm=0.15, du_max_finger=0.08)
  - Apply latency (ring buffer, delay 2 steps)
  - Map to actuator control ranges
    ↓
MuJoCo Physics Step (mj_step):
  - Forward kinematics
  - Dynamics simulation
  - Collision detection
  - Update state (qpos, qvel, site_xpos)
    ↓
_obs_now():
  - Get sensor data (EE pos, quat, touch, velocities, forces)
  - Get joint states (qpos, qvel)
  - Get privileged obs (if enabled): cube state, contact sum, plate z
  - Add observation noise (std=0.004)
  - Apply sensor delay (ring buffer, delay 1 step)
    ↓
_compute_reward():
  - Progress rewards: reach_prog, lift_prog (gated by contact)
  - Contact reward: touch_sum, n_fingers
  - Penalties: wrist_rate, jerk, control_mag, slip_torque
  - Step penalty: -0.002
  - Hold bonus: +10.0 nếu giữ thành công 120 steps
    ↓
Update curriculum level:
  - Nếu succ_ema > 0.35: tăng level (+0.002)
  - Nếu succ_ema <= 0.35: giảm level (-0.001)
  - Update domain randomization với level mới
    ↓
Check termination:
  - Safety: wrist_angvel > 20.0, NaN, invalid state
  - Success: hold >= 120 steps
  - Truncation: step_count >= max_steps
    ↓
Return (obs, rew, terminated, truncated, info)
```

### 4.4. Reward Shaping Logic

**Reward Components:**

1. **r_reach_prog** (max ~2.0):
   - Formula: `2.0 * (prev_dist - current_dist)`
   - Reward khi giảm khoảng cách đến cube

2. **r_lift_prog** (max ~1.0, gated):
   - Formula: `6.0 * (dz_rel - prev_dz_rel) * gate_contact`
   - Gate: `1.0 if touch_sum > 0.03 else 0.0`
   - Clipped: `[-0.5, 1.0]`
   - Reward khi nâng cube lên, chỉ khi đã có contact

3. **r_touch** (max ~0.8):
   - Formula: `0.4*tanh(0.6*touch_sum) + 0.4*(n_fingers>=2)*gate_contact`
   - Reward cho contact quality

4. **r_pen** (negative):
   - `-0.0015*wrist_rate`: Penalty cho angular velocity cao
   - `-0.0030*jerk`: Penalty cho control signal giật
   - `-0.0020*control_mag`: Penalty cho effort
   - `-0.0025*max(0, tau_mag - slip_torque_lim)`: Penalty cho slip risk

5. **r_step**: `-0.002` (khuyến khích hoàn thành nhanh)

6. **Hold bonus**: `+10.0` nếu giữ thành công 120 steps

**Total Reward**: `r_reach_prog + r_lift_prog + r_touch + r_pen + r_step + hold_bonus`

**Success Condition**: 
- `dz_rel > 0.04` (nâng lên 4cm)
- `touch_sum > 0.08` (đủ lực tiếp xúc)

### 4.5. MuJoCo Integration

**MuJoCo Model Structure:**
- **XML file**: `models/g1_dual_arm.xml`
  - Defines robot structure (bodies, joints, actuators)
  - Defines physics properties (masses, inertias, friction)
  - Defines control limits (forcerange, ctrlrange)
  - Defines sensors (EE pos, quat, touch, velocities, forces)
  
- **Actuators (22 total)**:
  - **Arm motors (14)**: Torque control với forcerange trong XML
  - **Finger synergy (10)**: Tendon-based control với ctrlrange trong XML
  - **Fist macros (2)**: Macro actuators cho left_fist, right_fist
  
- **Joints**:
  - **Arm joints (14)**: Shoulder, elbow, wrist joints
  - **Hand joints**: Finger MCP, PIP, DIP joints (không điều khiển trực tiếp)
  - **Cube free joint**: 7 DoF (3 pos + 4 quat) cho cube
  
- **Sensors**:
  - **EE positions**: `right_ee_pos`, `left_ee_pos`
  - **EE quaternions**: `right_ee_quat`, `left_ee_quat`
  - **Touch sensors**: `right_thumb_touch`, `right_index_touch`, ...
  - **Wrist velocities**: `right_wrist_vang`, `right_wrist_vlin`
  - **Palm forces/torques**: `right_palm_force`, `right_palm_torque`
  - **Cube/plate positions**: `cube_top_pos`, `plate_top_pos`

**Simulation Loop:**
```python
# 1. Set controls
for i, aid in enumerate(self.aid):
    if aid < 0: continue
    lo, hi = self.model.actuator_ctrlrange[aid]
    self.data.ctrl[aid] = lo + 0.5*(action[i]+1.0)*(hi-lo)

# 2. Step physics
mj.mj_step(self.model, self.data)

# 3. Read state
qpos = self.data.qpos  # Joint positions
qvel = self.data.qvel  # Joint velocities
sensordata = self.data.sensordata  # Sensor readings
```

### 4.6. Data Flow Diagram

```
┌─────────────────────────────────────────────────────────────┐
│                    Reinforcement Learning Loop               │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Agent (PPO Policy Network)                                 │
│  Input: Observation (50-70D, normalized)                     │
│  Output: Action (22D, normalized [-1, 1])                    │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  VecNormalize                                                │
│  - Normalize observations (running mean/std)                 │
│  - Normalize rewards (running mean/std)                     │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Environment (DualArmGraspEnvV22)                           │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  _set_action():                                      │   │
│  │  - Smoothing (EMA)                                   │   │
│  │  - Rate limiting                                     │   │
│  │  - Latency (ring buffer, 2 steps)                    │   │
│  │  - Map to actuator ranges                           │   │
│  └─────────────────────────────────────────────────────┘   │
│                              │                               │
│                              ▼                               │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  MuJoCo Physics (mj_step):                           │   │
│  │  - Forward kinematics                               │   │
│  │  - Dynamics simulation                              │   │
│  │  - Collision detection                             │   │
│  │  - Update state (qpos, qvel, sensordata)         │   │
│  └─────────────────────────────────────────────────────┘   │
│                              │                               │
│                              ▼                               │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  _obs_now():                                         │   │
│  │  - Get sensor data (EE, touch, velocities, forces)   │   │
│  │  - Get joint states (qpos, qvel)                   │   │
│  │  - Get privileged obs (cube, contact, plate)        │   │
│  │  - Add observation noise (std=0.004)                │   │
│  │  - Apply sensor delay (ring buffer, 1 step)          │   │
│  └─────────────────────────────────────────────────────┘   │
│                              │                               │
│                              ▼                               │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  _compute_reward():                                 │   │
│  │  - Progress rewards (reach, lift)                   │   │
│  │  - Contact reward                                   │   │
│  │  - Penalties (wrist_rate, jerk, effort, slip)      │   │
│  │  - Hold bonus (+10.0)                               │   │
│  └─────────────────────────────────────────────────────┘   │
│                              │                               │
│                              ▼                               │
│  ┌─────────────────────────────────────────────────────┐   │
│  │  Curriculum Learning:                                │   │
│  │  - Update level dựa trên success rate               │   │
│  │  - Update domain randomization                      │   │
│  └─────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
                    Return (obs, rew, done, info)
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Training Algorithm (PPO)                                    │
│  - Compute advantages (GAE)                                 │
│  - Update policy network (clipped)                          │
│  - Update value network                                     │
│  - Update VecNormalize statistics                           │
│  - Save checkpoints                                         │
└─────────────────────────────────────────────────────────────┘
```


## 5. Kết Luận và Nhận Xét

### 5.1. Tổng Kết Dự Án

Dự án **G1 Dual Arm Grasping RL (v22)** là một hệ thống hoàn chỉnh để huấn luyện robot hai tay thực hiện động tác nắm và nâng vật thể sử dụng Reinforcement Learning. Dự án này thể hiện:

1. **Tích hợp tốt các công nghệ**:
   - MuJoCo 3.x physics simulator
   - Gymnasium RL framework
   - Stable-Baselines3 RL algorithms (PPO)
   - PyTorch deep learning framework

2. **Thiết kế environment chuyên nghiệp**:
   - Custom Gymnasium environment với API chuẩn
   - Progress-based reward shaping với gating
   - Privileged observations cho critic (asymmetric)
   - Domain randomization với curriculum learning
   - Realistic delays (latency, sensor delay)

3. **Training infrastructure**:
   - VecNormalize để stabilize training
   - Parallel environments (SubprocVecEnv)
   - Checkpointing và resume training
   - Evaluation script chuẩn hóa

4. **Safety và robustness**:
   - Safety termination (wrist velocity, NaN checks)
   - Rate limiting và smoothing cho controls
   - Grip reflex để tránh slip
   - Domain randomization để tăng robustness

### 5.2. Điểm Mạnh

1. **Code organization tốt**:
   - Cấu trúc thư mục rõ ràng
   - Separation of concerns (env, training, evaluation)
   - Reusable components (common.py)

2. **Advanced features**:
   - Progress-based reward với gating
   - Privileged observations (asymmetric actor-critic)
   - Curriculum learning tự động
   - Domain randomization
   - Realistic delays (latency, sensor delay)

3. **Robustness**:
   - Noise augmentation (observation noise)
   - Safety mechanisms
   - Error handling
   - VecNormalize để stabilize training

4. **Documentation**:
   - README chi tiết với quickstart
   - Code comments rõ ràng
   - PowerShell scripts để dễ chạy

### 5.3. Điểm Cần Cải Thiện

1. **Demo script**:
   - Import sai (cần sửa `g1_dual_arm_env` → `g1_dual_arm_env_v22`)
   - Có thể thêm nhiều demo scenarios

2. **Evaluation**:
   - Có thể thêm visualization (trajectory plots, contact forces)
   - Có thể thêm metrics chi tiết hơn (grasp quality, stability)

3. **Training**:
   - Có thể thêm learning rate schedule
   - Có thể thêm early stopping
   - Có thể thêm TensorBoard logging chi tiết hơn

4. **Environment**:
   - Có thể thêm nhiều objects khác nhau (không chỉ cube)
   - Có thể thêm obstacles
   - Có thể thêm bimanual coordination tasks phức tạp hơn

### 5.4. Ứng Dụng Thực Tế

Dự án này có thể được mở rộng cho:

1. **Robotic manipulation**:
   - Object grasping và manipulation với nhiều objects
   - Hand-eye coordination
   - Bimanual coordination tasks phức tạp
   - Transfer learning sang real robot

2. **Research**:
   - Continuous control trong RL với privileged information
   - Curriculum learning strategies
   - Domain randomization techniques
   - Asymmetric actor-critic architectures

3. **Industrial applications**:
   - Pick and place tasks
   - Assembly tasks
   - Quality inspection với manipulation

### 5.5. Kết Luận

Dự án **G1 Dual Arm Grasping RL (v22)** là một implementation tốt của RL cho robotic manipulation tasks với contact-rich dynamics. Code được tổ chức tốt, dễ hiểu và mở rộng. Với reward shaping phù hợp, privileged observations, curriculum learning, và domain randomization, dự án này có thể serve như một baseline hoặc starting point cho các nghiên cứu và ứng dụng thực tế.

**Khuyến nghị**:
- Sử dụng dự án này như reference implementation cho manipulation tasks
- Mở rộng cho các tasks phức tạp hơn (multiple objects, obstacles)
- Thử nghiệm với các thuật toán RL khác (SAC, TD3, ...)
- Tích hợp với real robot hardware
- Thêm visualization và analysis tools

---


